<a href="https://colab.research.google.com/github/Shashankwer/Tensorflow_Testing/blob/master/GraphFunctions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to graphs and tf.function


### Graphs? 

Eager execution allows tensorflow to run operation by operation and returns the results back to Python. By Eager execution Tensorflow takes advantage of GPUs allowing to place variables, tensors and even operations on GPUs and TPUs. It is also easier to debug. 

Executing op by op in Python prevents the host acceleration which is otherwise available. If one can extract computation from Python one can make them into a graph. 

Graphs are the structures that contain a set of tf.Operation objects, which represents units of computation; and tf.Tensor object, which represent the units of data that flows between the operation. 

Graph are defined in a tf.Graph context. 

### The benefits of graphs: 

With graph one can have a great deal of flexibility. One can use Tensorflow graph in environments that do not have a Python interpreter, like mobile applications, embedded devices and backend services. Tensorflow uses graphs as the format for saved models when it exports them from python. 

Graphs are also easily optimized allowing the compiler to do the transformations like 

1. Statistically infer the value of the tensor by folding constatnnode in the computation
2. Seperate subparts of a computation that are independent and split them between threads or devices. 
3. Simplify arithematic operations by eliminating common subexpressions. 





In [1]:
import tensorflow as tf
import timeit
from datetime import datetime



In [2]:
def function_to_get_faster(x,y,b):
  x = tf.matmul(x,y)
  x = x + b
  return x

In [3]:
a_function_that_uses_graph = tf.function(function_to_get_faster)

In [4]:
x1 = tf.constant([[1.0,2.0]])
y1 = tf.constant([[2.0],[3.0]])
b1 = tf.constant(4.0)
a_function_that_uses_graph(x1,y1,b1).numpy()

array([[12.]], dtype=float32)

tf.function-ized functions are Python callbales and are equivalent to the python classes. They have a particular class 
`python.eager.def_function.Function` but to us its simply a non traced function version. 


In [5]:
def inner_function(x,y,b):
  #print(x,y,b)
  x = tf.matmul(x,y)
  x = x+b
  #print(x)
  return x


In [6]:
@tf.function
def outer_function(x):
  #print(x)
  y = tf.constant([[2.0],[3.0]])
  b = tf.constant(3.0)
  return inner_function(x,y,b)

#tf.function creates a graph.
outer_function(tf.constant([[1.0, 2.0]])).numpy()

array([[11.]], dtype=float32)

Flow Controls and side effects: 

Flow controls and loops are converted into Tensorflow via `tf.autograph` which makes use of methods standardizing loop constructs, unrolling and AST manipulation

In [7]:
def my_function(x):
  if tf.reduce_sum(x)<=1:
    return x*x
  else:
    return x-1
a_function = tf.function(my_function)
print("First Branch with graph",a_function(tf.constant(1.0)).numpy())
print("Second Branch with graph",a_function(tf.constant([5.0,6.0])).numpy())

First Branch with graph 1.0
Second Branch with graph [4. 5.]


In [8]:
print(tf.autograph.to_code(my_function))

def tf__my_function(x):
    with ag__.FunctionScope('my_function', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal do_return, retval_
            (do_return, retval_) = vars_

        def if_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = (ag__.ld(x) * ag__.ld(x))
            except:
                do_return = False
                raise

        def else_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = (ag__.ld(x) - 1)
            except:
                do_return = False
                raise
        ag__.if_stmt((ag__.converted_call(ag__.ld(tf).reduce_sum, (ag

### Seeing the speedup

Just wrapping a tensor-saped function `tf.function` does not automatically speeds up your code. For small functions called a few times on a single machine the overhead of calling a grpah or fragmented graph may dominate runtime. Also if most of the computation already happened on the accelerator, such as stacks of GPU heavy computation, the graph speedup wont be large.

For complicated computation, graph can provide a significant speedup. This is because graph can reduce the Python device to device communication and performs some speedups

In [9]:
class SequentialModel(tf.keras.Model):
  def __init__(self,**kwargs):
    super(SequentialModel,self).__init__(**kwargs)
    self.flatten = tf.keras.layers.Flatten(input_shape=(28,28))
    self.dense_1 = tf.keras.layers.Dense(128,activation='relu')
    self.dropout = tf.keras.layers.Dropout(0.2)
    self.dense_2 = tf.keras.layers.Dense(10)
  
  def call(self,x):
    x = self.flatten(x)
    x = self.dense_1(x)
    x = self.dropout(x)
    x = self.dense_2(x)
    return x


In [10]:
input_data = tf.random.uniform([60,28,28])
eager_model = SequentialModel()
graph_model = tf.function(eager_model)
print("Eager Time:", timeit.timeit(lambda: eager_model(input_data),number=10000))
print("Graph Time:", timeit.timeit(lambda: graph_model(input_data),number=10000))


Eager Time: 8.643591995999998
Graph Time: 6.703129660999991


## Polymorphic functions

A function stores the different kinds of datatyoe by which the grpah can be invoked. The function then stores the tf.Graph corresponding to the trace concrete_cuntion. If the function has already been traced with that kind of argument, one can just get the pretraced graph

Conceptually:
  1. A tf.Graph is the raw datastructure describing a computation
  2. A Function is a caching, tracing, dispatching over concrete function
  3. A ConcreteFunction is an eager compatible wrapped around a graph that allows to execute graph from Python

By default the tensorflow function execute with tf.Graph or with.Graph().as_default(). This measn that one is likely running in a graph context. Core functions in Tensorflow use graph contexts, such as Keras's model_fit()

For converting the function back to eager mode one can make use of:
1. Call models and layers directly as callables
2. When using Keras compile/fit at compile time use model.compile(run_eagerly=True)
3. Setting the global execution mode via tf.config.run_functions_eagerly(True)




In [11]:
class EagerLayer(tf.keras.layers.Layer):
  def __init__(self,**kwargs):
    super(EagerLayer,self).__init__(**kwargs)
    #some initialization
  
  def call(self,inputs):
    print("\nCurrently running eagerly",str(datetime.now))
    return inputs

In [12]:
class SequentialModel(tf.keras.Model):
  def __init__(self,**kwargs):
    super(SequentialModel,self).__init__(**kwargs)
    self.flatten = tf.keras.layers.Flatten(input_shape=(28,28))
    self.dense_1 = tf.keras.layers.Dense(128,activation='relu')
    self.dropout = tf.keras.layers.Dropout(0.2)
    self.dense_2 = tf.keras.layers.Dense(10)
    self.eager = EagerLayer()
  
  def call(self,x):
    x = self.flatten(x)
    x = self.dense_1(x)
    x = self.dropout(x)
    x = self.dense_2(x)
    return self.eager(x)

In [13]:
model = SequentialModel()

In [14]:
input_data = tf.random.uniform([60,28,28])
labels = tf.random.uniform([60])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [15]:
model.compile(run_eagerly=False,loss=loss_fn)

In [16]:
model.fit(input_data,labels,epochs=3)

Epoch 1/3

Currently running eagerly <built-in method now of type object at 0xa33e60>

Currently running eagerly <built-in method now of type object at 0xa33e60>
2/2 [==============================] - 0s 4ms/step - loss: 2.4185
Epoch 2/3
2/2 [==============================] - 0s 4ms/step - loss: 0.0100
Epoch 3/3
2/2 [==============================] - 0s 3ms/step - loss: 0.0049


In [17]:
print("Running eagerly")
model.compile(run_eagerly=True,loss=loss_fn)
model.fit(input_data,labels,epochs=3)

Running eagerly
Epoch 1/3

Currently running eagerly <built-in method now of type object at 0xa33e60>
1/2 [==============>...............] - ETA: 0s - loss: 0.0034
Currently running eagerly <built-in method now of type object at 0xa33e60>
2/2 [==============================] - 0s 10ms/step - loss: 0.0018
Epoch 2/3

Currently running eagerly <built-in method now of type object at 0xa33e60>
1/2 [==============>...............] - ETA: 0s - loss: 4.2431e-06
Currently running eagerly <built-in method now of type object at 0xa33e60>
2/2 [==============================] - 0s 9ms/step - loss: 6.0358e-06
Epoch 3/3

Currently running eagerly <built-in method now of type object at 0xa33e60>
1/2 [==============>...............] - ETA: 0s - loss: 6.6458e-06
Currently running eagerly <built-in method now of type object at 0xa33e60>
2/2 [==============================] - 0s 9ms/step - loss: 4.9809e-06


In [18]:
"""
Using run_functions_eagerly:

You can also globally set everything to run eagerly
"""

tf.config.run_functions_eagerly(True)
print("Running all functions eagerly")

polymorphic_functions = tf.function(model)

print(polymorphic_functions.get_concrete_function(input_data))

result = polymorphic_functions(input_data)
result = polymorphic_functions(input_data)


Running all functions eagerly

Currently running eagerly <built-in method now of type object at 0xa33e60>
ConcreteFunction function(self)
  Args:
    self: float32 Tensor, shape=(60, 28, 28)
  Returns:
    float32 Tensor, shape=(60, 10)

Currently running eagerly <built-in method now of type object at 0xa33e60>

Currently running eagerly <built-in method now of type object at 0xa33e60>


In [19]:
tf.config.experimental_run_functions_eagerly(False)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.
